In [14]:
import os
import numpy as np
from dotenv import load_dotenv
import random
import string
from pathlib import Path
import pandas as pd
from PIL import Image 
load_dotenv(override=True)

True

In [ ]:
BASE_DATA_DIR = Path(os.getenv("BASE_DATA_DIR"))
TARGET_DATA_DIR = Path("data")

#alphanumeric string for patient renaming
alphabet = string.ascii_lowercase + string.digits
patient_dict = {} #store patient name and id pairs
random.seed(1)

for patient in os.walk(BASE_DATA_DIR):
    if len(patient[2]) > 0: #only look at deepest directories which contain images
        full_path = Path(patient[0])
        angio_location = full_path.name
        angio_date = full_path.parent.name
        patient_name = full_path.parent.parent.name

        patient_id = ''.join(random.choices(alphabet, k=8))
        patient_dict[patient_id] = {"name": patient_name, "angio_date": angio_date, "angio_location": angio_location}
        
        labels_df = pd.DataFrame(columns=['leak', 'location', 'severity']) # create a labels df with the correct columns

        for i, frame in enumerate(patient[2]):
            new_img_name = f"frame{i+1}.png"
            current_img_path = full_path / frame
            save_path = TARGET_DATA_DIR / patient_id
            if not os.path.exists(save_path):
                os.mkdir(save_path)
            
            # TODO: make sure all images are the same size (even num channels)
            Image.open(current_img_path).save(save_path / new_img_name)
            
            # Create a new row with the frame name as the index
            labels = pd.DataFrame({"leak": [np.nan], "location": [np.nan], "severity": [np.nan]}, index=[new_img_name.split(".png")[0]])
            
            # Concatenate the new row to the labels_df
            labels_df = pd.concat([labels_df, labels])

        # Ensure the 'frame' is set as the index
        labels_df.index.name = 'frame'
        labels_df.to_csv(save_path / "labels.csv")

#save the file metadata to the data directory
pd.DataFrame(patient_dict).to_csv("data/metadata.csv")